In [9]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
import pandas as pd
import numpy as np
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup


# Initialize Selenium WebDriver for Firefox
firefox_options = Options()
# firefox_options.add_argument("--headless")  # Run headless Firefox to avoid opening a browser window
# service = FirefoxService(GeckoDriverManager().install())
driver = webdriver.Firefox(options=firefox_options)

# Open the web page
url = "https://www.imdb.com/chart/top/"  # Replace with the URL of the page you want to scrape
driver.get(url)

# Wait for the page to load completely (this time can be adjusted)
driver.implicitly_wait(10)  # seconds

#scroll to the bottom of the page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Get the page source after JavaScript has executed
html = driver.page_source

# Pass the HTML to BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Now you can use BeautifulSoup to parse the


The geckodriver version (0.33.0) detected in PATH at e:\Projects\Python\Movie_recommender\geckodriver.exe might not be compatible with the detected firefox version (125.0.2.8875); currently, geckodriver 0.34.0 is recommended for firefox 125.*, so it is advised to delete the driver in PATH and retry


In [32]:
#extracts links from first web
scraped_movies = soup.find_all('a' , class_ = 'ipc-title-link-wrapper')
list_scraped_movie = [str(i) for i in scraped_movies]
list_scraped_movie[:5]

['<a class="ipc-title-link-wrapper" href="/title/tt0111161/?ref_=chttp_t_1" tabindex="0"><h3 class="ipc-title__text">1. The Shawshank Redemption</h3></a>',
 '<a class="ipc-title-link-wrapper" href="/title/tt0068646/?ref_=chttp_t_2" tabindex="0"><h3 class="ipc-title__text">2. The Godfather</h3></a>',
 '<a class="ipc-title-link-wrapper" href="/title/tt0468569/?ref_=chttp_t_3" tabindex="0"><h3 class="ipc-title__text">3. The Dark Knight</h3></a>',
 '<a class="ipc-title-link-wrapper" href="/title/tt0071562/?ref_=chttp_t_4" tabindex="0"><h3 class="ipc-title__text">4. The Godfather Part II</h3></a>',
 '<a class="ipc-title-link-wrapper" href="/title/tt0050083/?ref_=chttp_t_5" tabindex="0"><h3 class="ipc-title__text">5. 12 Angry Men</h3></a>']

In [ ]:
#We Use regex to find links in strings
import re

def extract_url(html_string):
    match = re.search( r'href="([^"]+?)\?', html_string)
    return match.group(1)


extracted_urls = [extract_url(movie) for movie in list_scraped_movie]

extracted_urls


In [11]:
len(list_scraped_movie) #Only first 250 are usefuls

258

In [ ]:
Name_of_movies = [i.get_text() for i in scraped_movies][:250]
Name_of_movies

### We Create an example for first Movie ###
###(The Shawshank Redemption)###

In [41]:
#WE at first run it for an examole to see whats happen

import requests
HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
url_for_movie = 'https://www.imdb.com/' + extracted_urls[0] + 'plotsummary/?ref_=tt_stry_pl'
page_for_movie = requests.get(url_for_movie , headers = HEADERS)
content_for_movie =page_for_movie.text
soup_for_movie = BeautifulSoup(content_for_movie , 'html.parser')

In [ ]:
print(soup_for_movie.prettify())

In [45]:
m1 = soup_for_movie.find_all('div' , class_ = 'ipc-html-content ipc-html-content--base')

In [ ]:
m1.get_text()

In [ ]:
len(m1)

In [48]:
#The main Web scraping begins!!

data = []
for i in range(250):
    url_for_movie = 'https://www.imdb.com/' + extracted_urls[i] + 'plotsummary/?ref_=tt_stry_pl'
    page_for_movie = requests.get(url_for_movie , headers = HEADERS)
    content_for_movie =page_for_movie.text
    soup_for_movie = BeautifulSoup(content_for_movie , 'html.parser')

    m1 = soup_for_movie.find_all('div' , class_ = 'ipc-html-content ipc-html-content--base')
    data.append(m1[1].get_text().lower())

In [ ]:
len(data)

In [52]:
#We store it to a dataframe for saving in a excel file

df = pd.DataFrame(data = {'Name_of_movies': Name_of_movies , 'data' : data})
# df = pd.read_excel('imdb.xlsx')

In [53]:
df.head()

,Name_of_movies,data
0,1. The Shawshank Redemption,chronicles the experiences of a formerly succe...
1,2. The Godfather,"the godfather ""don"" vito corleone is the head ..."
2,3. The Dark Knight,set within a year after the events of batman b...
3,4. The Godfather Part II,the continuing saga of the corleone crime fami...
4,5. 12 Angry Men,"the defense and the prosecution have rested, a..."


### We Save that information on an excel file  for  backup ###

In [54]:
df.to_excel('imdb.xlsx' , index = False)

### Pre processing data (removing stop words , ...) ###

In [55]:
stopwords = 'a, an, and, are, as, at, be, but, by, for, if, in, into, is, it, no, not, of, on, or, such, that, the, their, then, there, these, they, this, to, was, will and with.'.split(', ')
special_charcters = ['`','~','!','@','#','$','%','^','&','*','(',')','_','-','+','=','{','[','}','}','|','\\', ':' , ';','"', "'",'<',',','>','.','?','//' ]
number = [str(i) for i in range(9)]

Idontwantthem = stopwords + special_charcters + number

In [ ]:
#Run it if you useing excel file that attach to it
data = list(df.data)

In [56]:
#We try to clean data from stopwords, special charcters and numbers
cleaned_data = []
for string in data:
    for unwanted in Idontwantthem:
        string = string.replace(unwanted , '')
    cleaned_data.append(string.split())

In [57]:
len(cleaned_data[0])

44

 ### TF_idf ###

In [70]:
All_words = set()
for j in cleaned_data:
   All_words = All_words.union({word for word in j})
All_words = list(All_words)
len(All_words)

6250

In [71]:
tf_idf = np.zeros(shape = (len(cleaned_data) , len(All_words)))

In [72]:
import math

def calculate_tf(word, document):

  return document.count(word) / len(document)


def calculate_idf(word, corpus):

  document_count = len(corpus)
  containing_documents = sum(1 for document in corpus if word in document)
  return math.log(document_count / (containing_documents + 1))



In [73]:
idfs = {}
for word in All_words:
    idfs[word] = calculate_idf(word , cleaned_data)


In [ ]:
idfs

In [75]:
t = 0
for i in range(len(cleaned_data)):
    for j in range(len(All_words)):
        tf_idf[i][j] = idfs[All_words[j]] * calculate_tf(All_words[j] , cleaned_data[i])

### KNN with cosine ###

In [77]:
def cosine_similarity(a , b):
    return np.dot(a , b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [81]:
cosine_similarity(np.random.rand(1,len(All_words)) , tf_idf[0])

array([0.07128307])

In [92]:
def KNN_with_cosine(a , tf_idf , Name_of_movies , k):
    distances = []
    for i in range(len(cleaned_data)):
        distances.append((cosine_similarity(a , tf_idf[i]) , i) )
    sorted_distance = sorted(distances , key = lambda x: -x[0])

    return [ Name_of_movies[j] for _,j in sorted_distance[:k]]



In [95]:
KNN_with_cosine(tf_idf[1] , tf_idf , Name_of_movies , 3)

['2. The Godfather', '4. The Godfather Part II', '196. The Deer Hunter']